In [4]:
from datetime import datetime, timedelta

import ipyplot
from dateutil.relativedelta import relativedelta
from IPython.display import Image
from pyinaturalist import (
    ICONIC_TAXA,
    Observation,
    TaxonCount,
    UserCount,
    enable_logging,
    get_observation_histogram,
    get_observation_identifiers,
    get_observation_observers,
    get_observation_species_counts,
    get_observations,
    pprint,
)
from rich import print

enable_logging()

In [11]:
import folium
import pandas as pd

In [6]:
def popup_html(df,row):
    name=df['taxon'].iloc[row] 
    place=df['place'].iloc[row] 
    url=df['url'].iloc[row]
    
    if df['EFG'].iloc[row] is None:
        html = """<!DOCTYPE html>
<html>
<head>
<h4 style="margin-bottom:10"; width="200px">{taxon}</h4>
</head>
<a href='{url}' target='inat'>iNat page</a>
    <table style="height: 126px; width: 200px;">
<tbody>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Place</span></td>
<td style="width: 60%;background-color: {color2};">{place}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Taxon</span></td>
<td style="width: 60%;background-color: {color2};">{taxon}</td>
</tr>
</tbody>
</table>
<span>{message}</span>
</html>
"""
        if df['has_context'].iloc[row] is None:
            message='This observation needs to be reviewed'
        elif df['has_context'].iloc[row] is True:
            message='This observation has been reviewed but some information is missing'
        else:
            message='This observation has been reviewed but does not have enough context to assign a Ecosystem Functional Group'
        
        output=html.format(taxon=name,place=place,url=url,EFG=None,comment=None,
                           color1="#a9a7bd",color2="#d2d0d3",message=message)

    else:
        efg=df['EFG'].iloc[row]
        comment=df['comment'].iloc[row]
    
        html = """<!DOCTYPE html>
<html>
<head>
<h4 style="margin-bottom:10"; width="200px">{taxon}</h4>
</head>
<a href='{url}' target='inat'>iNat page</a>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Place</span></td>
<td style="width: 60%;background-color: {color2};">{place}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Taxon</span></td>
<td style="width: 60%;background-color: {color2};">{taxon}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">EFG</span></td>
<td style="width: 60%;background-color: {color2};">{EFG}</td>
</tr>
<tr>
<td style="background-color: {color1};"><span style="color: #ffffff;">Comment</span></td>
<td style="width: 60%;background-color: {color2};">{comment}</td>
</tr>
</tbody>
</table>
</html>
"""
        output=html.format(taxon=name,place=place,url=url,EFG=efg,comment=comment,color1="#19a7bd",color2="#f2f0d3")
        
    return output

In [7]:

tids = [48039, # Lobelias
        68790, # Gorillas
        341182 # Dendrosenecio
       ]
tids = 341182
response=get_observations(place_id=7142, taxon_id=tids, page='all')
qry_observations = Observation.from_json_list(response)

[07-09 06:47:55] INFO     Request:                                             ]8;id=855158;file:///Users/jferrer/miniconda3/envs/ee/lib/python3.9/site-packages/pyinaturalist/session.py\session.py]8;;\:]8;id=782012;file:///Users/jferrer/miniconda3/envs/ee/lib/python3.9/site-packages/pyinaturalist/session.py#245\245]8;;\
                          GET https://api.inaturalist.org/v1/observations?taxo               
                          n_id=341182&place_id=7142&only_id=false&order_by=id&               
                          order=asc&per_page=200                                             
                          User-Agent: python-requests/2.25.1                                 
                          pyinaturalist/0.17.2                                               
                          Accept-Encoding: gzip, deflate                                     
                          Accept: application/json                                           
                          Connection: keep-alive                                             
                                                                                             

In [12]:
source = pd.DataFrame(
    [
        {
            'latitude': o.location[0],
            'longitude': o.location[1],
            'id':o.id,
            'url':o.uri,
            'place':o.place_guess,
            'taxon':o.species_guess,
            'iconic_taxon': o.taxon.iconic_taxon_name,
            'has_context':None,
            'reviewer':None,
            'EFG':None,
            'confidence':None,
            'comment':None,
        }
        for o in qry_observations
        if o.location
    ]
)

In [13]:
source.shape

(13, 12)

In [16]:
source.tail()

,latitude,longitude,id,url,place,taxon,iconic_taxon,has_context,reviewer,EFG,confidence,comment
8,-1.461997,29.488508,99692980,https://www.inaturalist.org/observations/99692980,"Musanze, Ruanda",Dendrosenecio erici-rosenii,Plantae,None,None,None,None,None
9,-1.461967,29.488538,99693416,https://www.inaturalist.org/observations/99693416,"Musanze, Ruanda",Dendrosenecio erici-rosenii,Plantae,None,None,None,None,None
10,-1.461887,29.488573,99693418,https://www.inaturalist.org/observations/99693418,"Musanze, Ruanda",Dendrosenecio erici-rosenii,Plantae,None,None,None,None,None
11,-1.387473,29.647607,106756629,https://www.inaturalist.org/observations/10675...,"Musanze, Rwanda",Dendrosenecio,Plantae,None,None,None,None,None
12,-1.387665,29.648567,106756635,https://www.inaturalist.org/observations/10675...,"Volcans, Musanze, Amajyaruguru, Rwanda",Dendrosenecio,Plantae,None,None,None,None,None


In [17]:
 # initialize the map and store it in a m object
m = folium.Map(location = [-2, 29.5],
               zoom_start = 8)

In [18]:
locations = source[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [19]:
for point in range(0, len(locationlist)):
    context=source['has_context'].iloc[point]
    if context is None:
        color = 'gray'
    elif context is True:
        color = 'darkblue'
    else:
        color = 'lightred'
    folium.Marker(locationlist[point], icon=folium.Icon(color=color), popup=popup_html(source,point)).add_to(m)

In [20]:
m

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;" data-html=%3C%21DOCTYPE%20html%3E%0A%3Chead%3E%20%20%20%20%0A%20%20%20%20%3Cmeta%20http-equiv%3D%22content-type%22%20content%3D%22text/html%3B%20charset%3DUTF-8%22%20/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%3Cscript%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20L_NO_TOUCH%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20L_DISABLE_3D%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%3C/script%3E%0A%20%20%20%20%0A%20%20%20%20%3Cstyle%3Ehtml%2C%20body%20%7Bwidth%3A%20100%25%3Bheight%3A%20100%25%3Bmargin%3A%200%3Bpadding%3A%200%3B%7D%3C/style%3E%0A%20%20%20%20%3Cstyle%3E%23map%20%7Bposition%3Aabsolute%3Btop%3A0%3Bbottom%3A0%3Bright%3A0%3Bleft%3A0%3B%7D%3C/style%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//code.jquery.com/jquery-1.12.4.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js%22%3E%3C/script%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css%22/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cmeta%20name%3D%22viewport%22%20content%3D%22width%3Ddevice-width%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20initial-scale%3D1.0%2C%20maximum-scale%3D1.0%2C%20user-scalable%3Dno%22%20/%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cstyle%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23map_d71ac99f368440ea83c91e52f3f98473%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20position%3A%20relative%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20width%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20left%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20top%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%3C/style%3E%0A%20%20%20%20%20%20%20%20%0A%3C/head%3E%0A%3Cbody%3E%20%20%20%20%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cdiv%20class%3D%22folium-map%22%20id%3D%22map_d71ac99f368440ea83c91e52f3f98473%22%20%3E%3C/div%3E%0A%20%20%20%20%20%20%20%20%0A%3C/body%3E%0A%3Cscript%3E%20%20%20%20%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20var%20map_d71ac99f368440ea83c91e52f3f98473%20%3D%20L.map%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22map_d71ac99f368440ea83c91e52f3f98473%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20center%3A%20%5B-2.0%2C%2029.5%5D%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20crs%3A%20L.CRS.EPSG3857%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20zoom%3A%208%2C%0A%20%20%20%20%20%20%20%20%2

In [ ]:
def evaluateContext(rid,context=None,validcoords=None):
    if context is None:
        input(...)